In [1]:
import pandas as pd 
import os 
import sys

sys.path.insert(0, os.getcwd() + '/code')
from viz import animate_play

In [2]:
df_game = pd.read_csv("data/games.csv")
df_tracking = pd.read_csv("data/week1.csv")
df_play = pd.read_csv("data/plays.csv")
df_players = pd.read_csv("data/players.csv")
df_pffScoutingData = pd.read_csv("data/pffScoutingData.csv")

In [3]:
animate_play(df_tracking,df_play,df_players,df_pffScoutingData,2021090900,97).show()